# Pandas Cheat Sheet

# Table of contents


- **The setup**: anaconda, Python, pandas, Jupyter
- **Importing data**: from csv (and options), from the web, creating from scratch, convering types, rename cols
- **Summarizing data**: len(df), shape, value_counts, head, tail, max(), min(), mean, dtype, info(), describe(), memory_usage(), scatter matrix, corr, isnull, notnull, unique(), nlargest
- **Selecting and computing**: select subset of row and cols, .loc, .iloc, drop columns, assign, apply/map/applymap, multiindex
- **Filtering and sorting**: >=, AND, OR, ==, ~, str.contains, str.startswith, sort_values, sort_index, filtering on sorted/unsorted, isin()
- **Split-apply-combine and pivots**: groupby, dt.month, dt.year, groupby.mean(), agg, stack, unstack, pivot, melt, merge
- **Time series manipulations**: downsampling, upsampling, rolling, mean, simple plotting
- **Plotting**: built-in plotting, advanced plotting, matplotlib, seaborn, styles, saving
- **Modeling and machine learning**: .value, feeding data, saving data
- **Misc tips and tricks**: pandas options, vectorization, timings with %%timeit, profiling with lprun

**principles:** small examples, no more than 5 rows. one or two data sets, no more.